In [79]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

import os
import os
import sys
import pandas as pd
from typing import List, Dict, Optional
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings

warnings.filterwarnings("ignore")
import plotly.express as px

In [80]:
df = pd.read_excel(r'C:\Users\TPWODL\New folder_Content\Twitter_X_Flow_Prediction_Tp\data\raw\twitter_x_data.xlsx')
dataset_path = r'C:\Users\TPWODL\New folder_Content\Twitter_X_Flow_Prediction_Tp\data\raw\twitter_x_data.xlsx'

In [81]:
df.shape

(32491, 23)

In [82]:
df.head(2)

,SL.NO,DATE,SHIFT DUTY,QUERY/REQUEST/COMPLAINT,COMPLAINT DETAILS,COMPLAINT NUMBER,SECTION,SUB-DIVISION,DIVISION,CIRCLE,...,DEPT,CLOSED/OPEN,REMARKS,TWEET-LINK,COMPLAINANT NAME,COMPLAINT RECEIVED TIME,RESPONSE TIME,SECOND RESPONSE TIME,FINAL RESPONSE TIME,FINAL RESPONSE DATE DD/MM/YYYY
0,1,2022-06-10,NaN,NaN,Mr. Mohan Maharana complaint regarding the fun...,NaN,NaN,NaN,NaN,NaN,...,O&M,Closed,"Dear Sir, Badchana is not coming under TPWODL ...",https://twitter.com/MohanMaharana4/status/1535...,NaN,NaN,NaN,NaN,NaN,NaN
1,2,2022-06-10,NaN,NaN,Mr.Bhawanispanda complaint regarding the repla...,4206102206136,SECTION-II,JUNAGARH,KWED,KALAHANDI,...,O&M,Closed,"Dear Sir, It is being taken up with the concer...",https://twitter.com/bhawanispanda/status/15351...,NaN,NaN,NaN,NaN,NaN,NaN


In [83]:
import pandas as pd

# Example: your column is called "TWEET-LINK"
col = df["TWEET-LINK"]

# Count DM directly
dm_count = (col == "DM").sum()

# Treat everything else as LINK
link_count = (col != "DM").sum()




In [84]:
df.loc[df["TWEET-LINK"] != "DM", "TWEET-LINK"].value_counts().sum()

np.int64(25009)

In [85]:
total_count = df["TWEET-LINK"].value_counts().sum()

In [86]:
total_count

np.int64(32491)

In [87]:
df.loc[df["TWEET-LINK"] != "DM", "TWEET-LINK"].value_counts().sum()

np.int64(25009)

In [88]:
(df["TWEET-LINK"].value_counts().sum(),df.loc[df["TWEET-LINK"] == "DM", "TWEET-LINK"].value_counts().sum(),
 df.loc[df["TWEET-LINK"] != "DM", "TWEET-LINK"].value_counts().sum()
)

(np.int64(32491), np.int64(7482), np.int64(25009))

In [89]:
# Ensure DATE column is in datetime format
df['DATE'] = pd.to_datetime(df['DATE'])

# Find the last (max) date
last_date = df['DATE'].max()

# Filter rows for that last date
last_day_rows = df[df['DATE'] == last_date]

# Get value counts for that date
counts = last_day_rows['DATE'].value_counts()

# If you want the total count (sum of occurrences)
total_count = counts.sum()

print("Last date:", last_date)
print("Counts on last date:\n", counts)
print("Total rows on last date:", total_count)


Last date: 2025-12-13 00:00:00
Counts on last date:
 DATE
2025-12-13    9
Name: count, dtype: int64
Total rows on last date: 9


In [99]:
def apply_pivot_table(dataset_path: str) -> pd.DataFrame:
    # Load dataset
    df = pd.read_excel(dataset_path)

    # Ensure required columns exist
    required = ['COMPLAINT TYPE', 'DEPT', 'CLOSED/OPEN']
    missing = [c for c in required if c not in df.columns]
    if missing:
        raise ValueError(f"Missing required columns: {missing}")

    # Work on a copy and clean text
    d = df[required].copy()

    # Normalize strings: strip, lower, then title-case
    def norm(s) -> str:
        if pd.isna(s):
            return '(blank)'
        s = str(s).strip()
        return '(blank)' if s == '' else s

    d['COMPLAINT TYPE'] = d['COMPLAINT TYPE'].map(norm)
    d['DEPT'] = d['DEPT'].map(norm)
    d['CLOSED/OPEN'] = d['CLOSED/OPEN'].map(norm)

    # Optional: unify common variants in complaint types
    # Map variations to a single canonical form
    type_map = {
        'Civil Works': 'Civil works',
        'NO Power Supply': 'No Power Supply',
        'Pole Shifting / Lt Sagging': 'Pole Shifting / Lt Sagging',
        'Transformer Failure / NCC': 'Transformer failure / NCC',
    }
    d['COMPLAINT TYPE'] = d['COMPLAINT TYPE'].replace(type_map)

    # Normalize DEPT values to desired set
    dept_map = {
        'Commercial': 'Commercial',
        'O&M': 'O&M',
        'Operation & Maintenance': 'O&M',
        'Other': 'Other',
        'Others': 'Other',
        '(blank)': '(blank)',
    }
    d['DEPT'] = d['DEPT'].map(lambda x: dept_map.get(x, x))

    # Normalize CLOSED/OPEN values to exactly CLOSED / OPEN
    status_map = {
        'Closed': 'CLOSED',
        'closed': 'CLOSED',
        'CLOSED': 'CLOSED',
        'OPEN': 'OPEN',
        'Open': 'OPEN',
        'open': 'OPEN',
        '(blank)': '(blank)',
    }
    d['CLOSED/OPEN'] = d['CLOSED/OPEN'].map(lambda x: status_map.get(x, x))

    # Build pivot with counts using size (do not set `values`)
    pivot = d.pivot_table(
        index='COMPLAINT TYPE',
        columns=['DEPT', 'CLOSED/OPEN'],
        aggfunc='size',
        fill_value=0
    )

    # Ensure consistent column order
    desired_depts = ['Commercial', 'O&M', 'Other', '(blank)']
    desired_status = ['CLOSED', 'OPEN']
    # Reindex columns to the desired multi-index order, keep missing combinations as 0
    pivot = pivot.reindex(
        pd.MultiIndex.from_product([desired_depts, desired_status], names=pivot.columns.names),
        axis=1,
        fill_value=0
    )

    # Add per-department TOTAL columns
    for dept in desired_depts:
        closed_col = (dept, 'CLOSED')
        open_col = (dept, 'OPEN')
        total_series = pivot.get(closed_col, 0) + pivot.get(open_col, 0)
        pivot[(dept, 'TOTAL')] = total_series

    # Add GRAND TOTAL across department TOTALs
    total_cols = [(dept, 'TOTAL') for dept in desired_depts]
    pivot[('Grand', 'TOTAL')] = pivot[total_cols].sum(axis=1)

    # Sort rows alphabetically (optional: or by Grand TOTAL descending)
    # pivot = pivot.sort_values(('Grand', 'TOTAL'), ascending=False)

    # Flatten columns to match your readability preference
    pivot.columns = [f"{lvl0} {lvl1}".strip() for lvl0, lvl1 in pivot.columns]

    # Reset index to have COMPLAINT TYPE as a column
    pivot = pivot.reset_index()
    
    # Add new columns for row-wise totals
    pivot["Grand Total Open"] = pivot[[col for col in pivot.columns if "OPEN" in col]].sum(axis=1)
    pivot["Grand Total Closed"] = pivot[[col for col in pivot.columns if "CLOSED" in col]].sum(axis=1)

    # Create total row
    total_row = {"COMPLAINT TYPE": "Grand Total"}
    
    # Sum each column except the first one
    for col in pivot.columns[1:]:
        total_row[col] = pivot[col].sum()

    # Append the total row
    pivot = pd.concat([pivot, pd.DataFrame([total_row])], ignore_index=True)

    return pivot

In [100]:
X_pivot = apply_pivot_table(dataset_path)

In [101]:
X_pivot

,COMPLAINT TYPE,Commercial CLOSED,Commercial OPEN,O&M CLOSED,O&M OPEN,Other CLOSED,Other OPEN,(blank) CLOSED,(blank) OPEN,Commercial TOTAL,O&M TOTAL,Other TOTAL,(blank) TOTAL,Grand TOTAL,Grand Total Open,Grand Total Closed
0,Billing,793,1,10,0,0,0,0,0,794,10,0,0,804,1,803
1,Civil works,0,0,1715,138,5,0,0,0,0,1853,5,0,1858,138,1720
2,Electrification,0,0,650,0,0,0,0,0,0,650,0,0,650,0,650
3,Low Voltage,0,0,1303,1,0,0,0,0,0,1304,0,0,1304,1,1303
4,Metering,190,2,1,0,0,0,0,0,192,1,0,0,193,2,191
5,NSC,730,3,6,0,3,0,0,0,733,6,3,0,742,3,739
6,No Power Supply,0,0,7695,2,23,0,0,0,0,7697,23,0,7720,2,7718
7,Others,0,0,382,0,4263,11,0,0,0,382,4274,0,4656,11,4645
8,Payment,138,0,0,0,1,0,0,0,138,0,1,0,139,0,139
9,Pole Shifting / Lt Sagging,0,0,964,68,1,0,0,0,0,1032,1,0,1033,68,965


In [130]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

def create_visualizations(pivot_df: pd.DataFrame):
    """
    Create interactive Plotly visualizations for complaint data
    
    Args:
        pivot_df: DataFrame returned from apply_pivot_examples function
    """
    
    # Remove the 'Total' row if it exists
    df = pivot_df[pivot_df['COMPLAINT TYPE'] != 'Total'].copy()
    
    # 1. STACKED BAR CHART - Best for comparing departments and status
    fig1 = go.Figure()
    
    departments = ['Commercial', 'O&M', 'Other']
    colors_closed = ['#10b981', '#059669', '#047857']
    colors_open = ['#ef4444', '#dc2626', '#b91c1c']
    
    for i, dept in enumerate(departments):
        closed_col = f'{dept} CLOSED'
        open_col = f'{dept} OPEN'
        
        if closed_col in df.columns:
            fig1.add_trace(go.Bar(
                name=f'{dept} - Closed',
                x=df['COMPLAINT TYPE'],
                y=df[closed_col],
                marker_color=colors_closed[i],
                hovertemplate='<b>%{x}</b><br>Department: ' + dept + '<br>Closed: %{y}<extra></extra>'
            ))
        
        if open_col in df.columns:
            fig1.add_trace(go.Bar(
                name=f'{dept} - Open',
                x=df['COMPLAINT TYPE'],
                y=df[open_col],
                marker_color=colors_open[i],
                hovertemplate='<b>%{x}</b><br>Department: ' + dept + '<br>Open: %{y}<extra></extra>'
            ))
    
    fig1.update_layout(
        title='Complaint Status by Type and Department',
        xaxis_title='Complaint Type',
        yaxis_title='Number of Complaints',
        xaxis=dict(tickangle=-45),
        barmode='stack',
        hovermode='x unified',
        template='plotly_white',
        height=600,
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
    )
    fig1.show()
    
    
    # 2. GROUPED BAR CHART - Closed vs Open comparison
    fig2 = go.Figure()
    
    total_closed = df['Total Closed'] if 'Total Closed' in df.columns else df[[col for col in df.columns if 'CLOSED' in col]].sum(axis=1)
    total_open = df['Total Open'] if 'Total Open' in df.columns else df[[col for col in df.columns if 'OPEN' in col]].sum(axis=1)
    
    fig2.add_trace(go.Bar(
        name='Closed',
        x=df['COMPLAINT TYPE'],
        y=total_closed,
        marker_color='#10b981',
        text=total_closed,
        textposition='auto',
    ))
    
    fig2.add_trace(go.Bar(
        name='Open',
        x=df['COMPLAINT TYPE'],
        y=total_open,
        marker_color='#ef4444',
        text=total_open,
        textposition='auto',
    ))
    
    fig2.update_layout(
        title='Total Closed vs Open Complaints by Type',
        xaxis_title='Complaint Type',
        yaxis_title='Count',
        xaxis=dict(tickangle=-45),
        barmode='group',
        template='plotly_white',
        height=500,
        showlegend=True
    )
    fig2.show()
    
    
    # 3. PIE CHART - Overall Status Distribution
    total_closed_sum = total_closed.sum()
    total_open_sum = total_open.sum()
    
    fig3 = go.Figure(data=[go.Pie(
        labels=['Closed', 'Open'],
        values=[total_closed_sum, total_open_sum],
        hole=0.4,
        marker_colors=['#10b981', '#ef4444'],
        textinfo='label+percent+value',
        textfont_size=14
    )])
    
    fig3.update_layout(
        title='Overall Complaint Status Distribution',
        template='plotly_white',
        height=400
    )
    fig3.show()
    
    
    # 4. SUNBURST CHART - Hierarchical view (Best for seeing relationships)
    sunburst_data = []
    
    for idx, row in df.iterrows():
        complaint_type = row['COMPLAINT TYPE']
        
        for dept in departments:
            closed_col = f'{dept} CLOSED'
            open_col = f'{dept} OPEN'
            
            if closed_col in df.columns and row[closed_col] > 0:
                sunburst_data.append({
                    'complaint': complaint_type,
                    'department': dept,
                    'status': 'Closed',
                    'value': row[closed_col]
                })
            
            if open_col in df.columns and row[open_col] > 0:
                sunburst_data.append({
                    'complaint': complaint_type,
                    'department': dept,
                    'status': 'Open',
                    'value': row[open_col]
                })
    
    if sunburst_data:
        sunburst_df = pd.DataFrame(sunburst_data)
        
        fig4 = px.sunburst(
            sunburst_df,
            path=['complaint', 'department', 'status'],
            values='value',
            color='status',
            color_discrete_map={'Closed': '#10b981', 'Open': '#ef4444'},
            title='Complaint Hierarchy: Type → Department → Status'
        )
        fig4.update_layout(height=600)
        fig4.show()
    
    
    # 5. HEATMAP - Best for spotting patterns
    heatmap_data = []
    
    for dept in departments:
        dept_total = []
        for idx, row in df.iterrows():
            closed = row.get(f'{dept} CLOSED', 0)
            open_val = row.get(f'{dept} OPEN', 0)
            dept_total.append(closed + open_val)
        heatmap_data.append(dept_total)
    
    fig5 = go.Figure(data=go.Heatmap(
        z=heatmap_data,
        x=df['COMPLAINT TYPE'],
        y=departments,
        colorscale='Viridis',
        text=heatmap_data,
        texttemplate='%{text}',
        textfont={"size": 12},
        hovertemplate='Department: %{y}<br>Complaint: %{x}<br>Count: %{z}<extra></extra>'
    ))
        
    fig5.update_layout(
        title='Complaint Distribution Heatmap',
        xaxis_title='Complaint Type',
        yaxis_title='Department',
        xaxis=dict(tickangle=-45),
        template='plotly_white',   # base dark theme
        height=400
    )
    fig5.show()


    
        
    if sunburst_data:
        fig6 = px.treemap(
            sunburst_df,
            path=['department', 'complaint', 'status'],
            values='value',
            color='status',
            color_discrete_map={
                'Closed': '#3b82f6',   # blue
                'Open': '#f59e0b'      # amber/orange
            },
            title='Complaint Treemap: Department → Type → Status'
        )

        fig6.update_layout(
            height=600,
            template="plotly_white",              # clean base style           
        )

        fig6.show()


In [131]:
fig = create_visualizations(X_pivot)


In [122]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

def create_visualizations(pivot_df: pd.DataFrame):
    """
    Create interactive Plotly visualizations for complaint data
    with distinct color schemes and backgrounds
    """
    
    # Remove the 'Total' row if it exists
    df = pivot_df[pivot_df['COMPLAINT TYPE'] != 'Total'].copy()
    
    # 1. STACKED BAR CHART - teal & red on light gray
    fig1 = go.Figure()
    
    departments = ['Commercial', 'O&M', 'Other']
    colors_closed = ['#14b8a6', '#0d9488', '#115e59']   # teal shades
    colors_open = ['#f87171', '#dc2626', '#991b1b']     # red shades
    
    for i, dept in enumerate(departments):
        closed_col = f'{dept} CLOSED'
        open_col = f'{dept} OPEN'
        
        if closed_col in df.columns:
            fig1.add_trace(go.Bar(
                name=f'{dept} - Closed',
                x=df['COMPLAINT TYPE'],
                y=df[closed_col],
                marker_color=colors_closed[i],
                hovertemplate='<b>%{x}</b><br>Department: ' + dept + '<br>Closed: %{y}<extra></extra>'
            ))
        
        if open_col in df.columns:
            fig1.add_trace(go.Bar(
                name=f'{dept} - Open',
                x=df['COMPLAINT TYPE'],
                y=df[open_col],
                marker_color=colors_open[i],
                hovertemplate='<b>%{x}</b><br>Department: ' + dept + '<br>Open: %{y}<extra></extra>'
            ))
    
    fig1.update_layout(
        title='Complaint Status by Type and Department',
        xaxis_title='Complaint Type',
        yaxis_title='Number of Complaints',
        xaxis=dict(tickangle=-45),
        barmode='stack',
        hovermode='x unified',
        template='plotly_white',
        plot_bgcolor="rgb(240,240,240)",   # light gray background
        paper_bgcolor="rgb(240,240,240)",
        height=600,
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
    )
    fig1.show()
    
    
    # 2. GROUPED BAR CHART - purple vs orange on dark background
    fig2 = go.Figure()
    
    total_closed = df['Total Closed'] if 'Total Closed' in df.columns else df[[col for col in df.columns if 'CLOSED' in col]].sum(axis=1)
    total_open = df['Total Open'] if 'Total Open' in df.columns else df[[col for col in df.columns if 'OPEN' in col]].sum(axis=1)
    
    fig2.add_trace(go.Bar(
        name='Closed',
        x=df['COMPLAINT TYPE'],
        y=total_closed,
        marker_color='#8b5cf6',   # violet
        text=total_closed,
        textposition='auto',
    ))
    
    fig2.add_trace(go.Bar(
        name='Open',
        x=df['COMPLAINT TYPE'],
        y=total_open,
        marker_color='#f97316',   # orange
        text=total_open,
        textposition='auto',
    ))
    
    fig2.update_layout(
        title='Total Closed vs Open Complaints by Type',
        xaxis_title='Complaint Type',
        yaxis_title='Count',
        xaxis=dict(tickangle=-45),
        barmode='group',
        template='plotly_dark',
        plot_bgcolor="rgb(40,40,40)",   # dark gray
        paper_bgcolor="rgb(25,25,25)",  # near black
        height=500,
        showlegend=True
    )
    fig2.show()
    
    
    # 3. PIE CHART - green vs pink on off-white
    total_closed_sum = total_closed.sum()
    total_open_sum = total_open.sum()
    
    fig3 = go.Figure(data=[go.Pie(
        labels=['Closed', 'Open'],
        values=[total_closed_sum, total_open_sum],
        hole=0.4,
        marker_colors=['#22c55e', '#ec4899'],   # green vs pink
        textinfo='label+percent+value',
        textfont_size=14
    )])
    
    fig3.update_layout(
        title='Overall Complaint Status Distribution',
        template='plotly_white',
        plot_bgcolor="rgb(250,250,250)",   # off-white
        paper_bgcolor="rgb(250,250,250)",
        height=400
    )
    fig3.show()
    
    
    # 4. SUNBURST CHART - blue vs amber on light theme
    sunburst_data = []
    for idx, row in df.iterrows():
        complaint_type = row['COMPLAINT TYPE']
        for dept in departments:
            closed_col = f'{dept} CLOSED'
            open_col = f'{dept} OPEN'
            if closed_col in df.columns and row[closed_col] > 0:
                sunburst_data.append({'complaint': complaint_type, 'department': dept, 'status': 'Closed', 'value': row[closed_col]})
            if open_col in df.columns and row[open_col] > 0:
                sunburst_data.append({'complaint': complaint_type, 'department': dept, 'status': 'Open', 'value': row[open_col]})
    
    if sunburst_data:
        sunburst_df = pd.DataFrame(sunburst_data)
        fig4 = px.sunburst(
            sunburst_df,
            path=['complaint', 'department', 'status'],
            values='value',
            color='status',
            color_discrete_map={'Closed': '#3b82f6', 'Open': '#f59e0b'},  # blue vs amber
            title='Complaint Hierarchy: Type → Department → Status'
        )
        fig4.update_layout(
            height=600,
            template="plotly_white",
            plot_bgcolor="rgb(245,245,245)",
            paper_bgcolor="rgb(245,245,245)"
        )
        fig4.show()
    
    
    # 5. HEATMAP - purple-green palette on dark gray
    heatmap_data = []
    for dept in departments:
        dept_total = []
        for idx, row in df.iterrows():
            closed = row.get(f'{dept} CLOSED', 0)
            open_val = row.get(f'{dept} OPEN', 0)
            dept_total.append(closed + open_val)
        heatmap_data.append(dept_total)
    
    fig5 = go.Figure(data=go.Heatmap(
        z=heatmap_data,
        x=df['COMPLAINT TYPE'],
        y=departments,
        colorscale='Magma',   # purple-orange gradient
        text=heatmap_data,
        texttemplate='%{text}',
        textfont={"size": 12},
        hovertemplate='Department: %{y}<br>Complaint: %{x}<br>Count: %{z}<extra></extra>'
    ))
        
    fig5.update_layout(
        title='Complaint Distribution Heatmap',
        xaxis_title='Complaint Type',
        yaxis_title='Department',
        xaxis=dict(tickangle=-45),
        template='plotly_dark',
        plot_bgcolor="rgb(35,35,35)",   # dark gray
        paper_bgcolor="rgb(20,20,20)",  # near black
        height=400
    )
    fig5.show()
    
    
    # 6. TREEMAP - teal vs orange on off-white
    if sunburst_data:
        fig6 = px.treemap(
            sunburst_df,
            path=['department', 'complaint', 'status'],
            values='value',
            color='status',
            color_discrete_map={'Closed': '#0ea5e9', 'Open': '#f97316'},  # teal vs orange
            title='Complaint Treemap: Department → Type → Status'
        )
        fig6.update_layout(
            height=600,
            template="plotly_white",
            plot_bgcolor="rgb(245,245,245)",
            paper_bgcolor="rgb(245,245,245)"
        )
        fig6.show()


In [123]:
fig = create_visualizations(X_pivot)
